In [10]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from google.colab import drive
import glob
from matplotlib import pyplot as plt
from PIL import Image
import os
from os import path

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
generate_dataset(
  dataset_path = '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Datasets/Full Dataset/AMR 5 New',
  source_dataset_path = '/content/drive/MyDrive/Mobile Robotics Project/Full Dataset/ABF',
  image_enhancement = adaptive_multiscale_retinex_5
)

Generating dataset...

Dataset Path : /content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Datasets/Full Dataset/AMR 5 New
Source Dataset Path : /content/drive/MyDrive/Mobile Robotics Project/Full Dataset/ABF
Image Enhancement : <function adaptive_multiscale_retinex_5 at 0x7f029be3b310>

Index 0/991...
Index 20/991...
Index 40/991...
Index 60/991...
Index 80/991...
Index 100/991...
Index 120/991...
Index 140/991...
Index 160/991...
Index 180/991...
Index 200/991...
Index 220/991...
Index 240/991...
Index 260/991...
Index 280/991...
Index 300/991...
Index 320/991...
Index 340/991...
Index 360/991...
Index 380/991...
Index 400/991...
Index 420/991...
Index 440/991...
Index 460/991...
Index 480/991...
Index 500/991...
Index 520/991...
Index 540/991...
Index 560/991...
Index 580/991...
Index 600/991...
Index 620/991...
Index 640/991...
Index 660/991...
Index 680/991...
Index 700/991...
Index 720/991...
Index 740/991...
Index 760/991...
Index 780/991...
Index 800/991...
Index 82

In [15]:
# Dataset generator method
def generate_dataset(dataset_path, source_dataset_path, image_enhancement, verbose=True):

  # Output header
  header = 'Generating dataset...\n\n'
  header += f'Dataset Path : {dataset_path}\n'
  header += f'Source Dataset Path : {source_dataset_path}\n'
  header += f'Image Enhancement : {image_enhancement}\n'
  print(header)

  # Create dataset folders
  if not path.exists(f'{dataset_path}'):
    os.mkdir(f'{dataset_path}')
  if not path.exists(f'{dataset_path}/image_0'):
    os.mkdir(f'{dataset_path}/image_0')
  if not path.exists(f'{dataset_path}/image_1'):
    os.mkdir(f'{dataset_path}/image_1')

  # Method for getting the index from a dataset path
  def get_index_from_path(path):
    if 'image_0' in path:
      left_text = f'{source_dataset_path}/image_0/'
    elif 'image_1' in path:
      left_text = f'{source_dataset_path}/image_1/'
    right_text = '.png'
    left_idx = path.index(left_text)
    right_idx = path.index(right_text)
    res = ''
    for idx in range(left_idx + len(left_text), right_idx):
        res = res + path[idx]
    return res

  # Loop through all source dataset images
  image_paths = zip( sorted(glob.glob(f'{source_dataset_path}/image_0/*.png')) , sorted(glob.glob(f'{source_dataset_path}/image_1/*.png')) )
  for i, (image_0_path, image_1_path) in enumerate(image_paths):

    # Generate image_0 image
    idx = get_index_from_path(image_0_path)
    if not os.path.isfile(f'{dataset_path}/image_0/{idx}.png'):
      img = cv2.imread(image_0_path)
      img = image_enhancement(img)
      cv2.imwrite(f'{dataset_path}/image_0/{idx}.png', img)

  # Generate image_1 image
    idx = get_index_from_path(image_1_path)
    if not os.path.isfile(f'{dataset_path}/image_1/{idx}.png'):
      img = cv2.imread(image_1_path)
      img = image_enhancement(img)
      cv2.imwrite(f'{dataset_path}/image_1/{idx}.png', img)

    # Print progress
    if verbose and i % 20 == 0:
      print(f'Index {i}/{len(glob.glob(f"{source_dataset_path}/image_0/*.png"))}...')

  # # Copy other dataset files
  # %cp -av '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Dataset Files/tartanair.yaml' '{dataset_path}/'
  # %cp -av '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Dataset Files/tartanair2.yaml' '{dataset_path}/'
  # %cp -av '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Dataset Files/tartanair3.yaml' '{dataset_path}/'
  # %cp -av '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Dataset Files/tartanair.yamlZone.Identifier' '{dataset_path}/'
  # %cp -av '/content/drive/MyDrive/Mobile Robotics Project/Image Enhancement/Dataset Files/times.txt' '{dataset_path}/'

In [ ]:
# Team 22

def team_22_1(img):

  src = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  res = np.zeros(src.shape, np.uint8)
  gui_small, gui_large = cv2.ximgproc.guidedFilter(src, src, 2, 0.1*0.1), cv2.ximgproc.guidedFilter(src, src, 2*10, 0.1*0.1)
  residual = gui_small - gui_large
  detail = gui_small + 5 * residual
  clahe = cv2.createCLAHE(clipLimit=8, tileGridSize=(8,8))
  res = clahe.apply(detail)

  return res

def team_22_color_1(img):

  res = np.zeros(img.shape[:2], dtype=np.uint8)
  gui_small = cv2.ximgproc.guidedFilter(img, img, 2, 0.1*0.1)
  gui_large = cv2.ximgproc.guidedFilter(img, img, 2*10, 0.1*0.1)
  residual = gui_small - gui_large
  detail = gui_small + 5 * residual
  lab = cv2.cvtColor(detail, cv2.COLOR_BGR2LAB)
  lab_planes = list(cv2.split(lab))
  clahe = cv2.createCLAHE(clipLimit=8.0, tileGridSize=(8,8))
  lab_planes[0] = clahe.apply(lab_planes[0])
  lab = cv2.merge(tuple(lab_planes))
  res = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

  return res

In [ ]:
# Adaptive Histogram Equalization

def adaptive_histogram_equalization_1(img, clipLimit=2.0, tileGridSize=(8,8)):

  # Convert input image to LAB color space
  lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

  # Split the L, A, and B channels of the LAB image
  l_channel, a, b = cv2.split(lab)

  # Create a CLAHE object with a clip limit of 2.0 and a tile size of 8x8
  clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)

  # Apply CLAHE to the L channel of the LAB image
  cl = clahe.apply(l_channel)

  # Merge the enhanced L channel with the original A and B channels
  limg = cv2.merge((cl,a,b))

  # Convert the LAB image back to the BGR color space
  enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

  return enhanced_img

def adaptive_histogram_equalization_2(img, clipLimit=4.0, tileGridSize=(8,8)):

  # Convert input image to LAB color space
  lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

  # Split the L, A, and B channels of the LAB image
  l_channel, a, b = cv2.split(lab)

  # Create a CLAHE object with a clip limit of 2.0 and a tile size of 8x8
  clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)

  # Apply CLAHE to the L channel of the LAB image
  cl = clahe.apply(l_channel)

  # Merge the enhanced L channel with the original A and B channels
  limg = cv2.merge((cl,a,b))

  # Convert the LAB image back to the BGR color space
  enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

  return enhanced_img

def adaptive_histogram_equalization_3(img, clipLimit=2.0, tileGridSize=(4,4)):

  # Convert input image to LAB color space
  lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

  # Split the L, A, and B channels of the LAB image
  l_channel, a, b = cv2.split(lab)

  # Create a CLAHE object with a clip limit of 2.0 and a tile size of 8x8
  clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)

  # Apply CLAHE to the L channel of the LAB image
  cl = clahe.apply(l_channel)

  # Merge the enhanced L channel with the original A and B channels
  limg = cv2.merge((cl,a,b))

  # Convert the LAB image back to the BGR color space
  enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

  return enhanced_img

def adaptive_histogram_equalization_4(img, clipLimit=4.0, tileGridSize=(4,4)):

  # Convert input image to LAB color space
  lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

  # Split the L, A, and B channels of the LAB image
  l_channel, a, b = cv2.split(lab)

  # Create a CLAHE object with a clip limit of 2.0 and a tile size of 8x8
  clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)

  # Apply CLAHE to the L channel of the LAB image
  cl = clahe.apply(l_channel)

  # Merge the enhanced L channel with the original A and B channels
  limg = cv2.merge((cl,a,b))

  # Convert the LAB image back to the BGR color space
  enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

  return enhanced_img

def adaptive_histogram_equalization_5(img, clipLimit=4.0, tileGridSize=(2,2)):

  # Convert input image to LAB color space
  lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

  # Split the L, A, and B channels of the LAB image
  l_channel, a, b = cv2.split(lab)

  # Create a CLAHE object with a clip limit of 2.0 and a tile size of 8x8
  clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)

  # Apply CLAHE to the L channel of the LAB image
  cl = clahe.apply(l_channel)

  # Merge the enhanced L channel with the original A and B channels
  limg = cv2.merge((cl,a,b))

  # Convert the LAB image back to the BGR color space
  enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

  return enhanced_img

In [ ]:
# Adaptive multiscale retinex

def adaptive_multiscale_retinex_1(image, sigma_list=[15, 80, 250], alpha=125, beta=46, gamma=15, gain=1):

    # Convert image to float32 for processing
    image = np.float32(image)

    # Compute the logarithm of the image
    log_image = np.log10(image + 1.0)

    # Initialize the output image
    enhanced_image = np.zeros_like(image)

    # Perform AMSR on each scale
    for sigma in sigma_list:
        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(log_image, (0, 0), sigma)
        # Compute the MSR component
        msr_component = log_image - blurred_image
        # Amplify the MSR component
        msr_component = alpha * msr_component
        # Normalize the MSR component
        msr_component = cv2.normalize(msr_component, None, 0, beta, cv2.NORM_MINMAX)
        # Exponentiate the MSR component
        msr_component = np.power(10, msr_component)
        # Add the MSR component to the enhanced image
        enhanced_image += msr_component

    # Normalize the enhanced image
    enhanced_image = cv2.normalize(enhanced_image, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the enhanced image back to uint8
    enhanced_image = np.uint8(enhanced_image)

    # Adjust the enhancement level with the gain factor
    enhanced_image = cv2.addWeighted(image, gain, enhanced_image, 1 - gain, 0, dtype=cv2.CV_32F) # Explicitly specify the data type of the output image

    return enhanced_image

def adaptive_multiscale_retinex_2(image, sigma_list=[10, 25, 50, 100, 150, 200, 250], alpha=125, beta=46, gamma=15, gain=1):

    # Convert image to float32 for processing
    image = np.float32(image)

    # Compute the logarithm of the image
    log_image = np.log10(image + 1.0)

    # Initialize the output image
    enhanced_image = np.zeros_like(image)

    # Perform AMSR on each scale
    for sigma in sigma_list:
        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(log_image, (0, 0), sigma)
        # Compute the MSR component
        msr_component = log_image - blurred_image
        # Amplify the MSR component
        msr_component = alpha * msr_component
        # Normalize the MSR component
        msr_component = cv2.normalize(msr_component, None, 0, beta, cv2.NORM_MINMAX)
        # Exponentiate the MSR component
        msr_component = np.power(10, msr_component)
        # Add the MSR component to the enhanced image
        enhanced_image += msr_component

    # Normalize the enhanced image
    enhanced_image = cv2.normalize(enhanced_image, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the enhanced image back to uint8
    enhanced_image = np.uint8(enhanced_image)

    # Adjust the enhancement level with the gain factor
    enhanced_image = cv2.addWeighted(image, gain, enhanced_image, 1 - gain, 0, dtype=cv2.CV_32F) # Explicitly specify the data type of the output image

    return enhanced_image

def adaptive_multiscale_retinex_3(image, sigma_list=list(np.linspace(5,255,21).astype(int)), alpha=125, beta=46, gamma=15, gain=1):

    # Convert image to float32 for processing
    image = np.float32(image)

    # Compute the logarithm of the image
    log_image = np.log10(image + 1.0)

    # Initialize the output image
    enhanced_image = np.zeros_like(image)

    # Perform AMSR on each scale
    for sigma in sigma_list:
        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(log_image, (0, 0), sigma)
        # Compute the MSR component
        msr_component = log_image - blurred_image
        # Amplify the MSR component
        msr_component = alpha * msr_component
        # Normalize the MSR component
        msr_component = cv2.normalize(msr_component, None, 0, beta, cv2.NORM_MINMAX)
        # Exponentiate the MSR component
        msr_component = np.power(10, msr_component)
        # Add the MSR component to the enhanced image
        enhanced_image += msr_component

    # Normalize the enhanced image
    enhanced_image = cv2.normalize(enhanced_image, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the enhanced image back to uint8
    enhanced_image = np.uint8(enhanced_image)

    # Adjust the enhancement level with the gain factor
    enhanced_image = cv2.addWeighted(image, gain, enhanced_image, 1 - gain, 0, dtype=cv2.CV_32F) # Explicitly specify the data type of the output image

    return enhanced_image

def adaptive_multiscale_retinex_4(image, sigma_list=[10, 25, 50, 100, 150, 200, 250], alpha=200, beta=75, gamma=20, gain=1):

    # Convert image to float32 for processing
    image = np.float32(image)

    # Compute the logarithm of the image
    log_image = np.log10(image + 1.0)

    # Initialize the output image
    enhanced_image = np.zeros_like(image)

    # Perform AMSR on each scale
    for sigma in sigma_list:
        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(log_image, (0, 0), sigma)
        # Compute the MSR component
        msr_component = log_image - blurred_image
        # Amplify the MSR component
        msr_component = alpha * msr_component
        # Normalize the MSR component
        msr_component = cv2.normalize(msr_component, None, 0, beta, cv2.NORM_MINMAX)
        # Exponentiate the MSR component
        msr_component = np.power(10, msr_component)
        # Add the MSR component to the enhanced image
        enhanced_image += msr_component

    # Normalize the enhanced image
    enhanced_image = cv2.normalize(enhanced_image, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the enhanced image back to uint8
    enhanced_image = np.uint8(enhanced_image)

    # Adjust the enhancement level with the gain factor
    enhanced_image = cv2.addWeighted(image, gain, enhanced_image, 1 - gain, 0, dtype=cv2.CV_32F) # Explicitly specify the data type of the output image

    return enhanced_image

def adaptive_multiscale_retinex_5(image, sigma_list=[10, 25, 50, 100, 150, 200, 250], alpha=100, beta=35, gamma=10, gain=1):

    # Convert image to float32 for processing
    image = np.float32(image)

    # Compute the logarithm of the image
    log_image = np.log10(image + 1.0)

    # Initialize the output image
    enhanced_image = np.zeros_like(image)

    # Perform AMSR on each scale
    for sigma in sigma_list:
        # Apply Gaussian blur
        blurred_image = cv2.GaussianBlur(log_image, (0, 0), sigma)
        # Compute the MSR component
        msr_component = log_image - blurred_image
        # Amplify the MSR component
        msr_component = alpha * msr_component
        # Normalize the MSR component
        msr_component = cv2.normalize(msr_component, None, 0, beta, cv2.NORM_MINMAX)
        # Exponentiate the MSR component
        msr_component = np.power(10, msr_component)
        # Add the MSR component to the enhanced image
        enhanced_image += msr_component

    # Normalize the enhanced image
    enhanced_image = cv2.normalize(enhanced_image, None, 0, 255, cv2.NORM_MINMAX)
    # Convert the enhanced image back to uint8
    enhanced_image = np.uint8(enhanced_image)

    # Adjust the enhancement level with the gain factor
    enhanced_image = cv2.addWeighted(image, gain, enhanced_image, 1 - gain, 0, dtype=cv2.CV_32F) # Explicitly specify the data type of the output image

    return enhanced_image